In [20]:
import pandas as pd
import os
import sys
home = os.path.expanduser('~')
my_packages = os.path.join(home, 'my_packages')
sys.path.append(my_packages)
import tree_structures as ts
import json
import dfs_sort_utils
import date_utils
import taxonomy_utils

<span style="font-family:Papyrus; font-size:1.5em;">EDA of kadarbitr taxonomy.</span>

In [2]:
data = json.load(open('kad.arbitr-parser/data.json', 'r'))
kada_cats = list(data.keys())
assert dfs_sort_utils.dfs_sort(kada_cats) == kada_cats, 'categories are not in dfs order'
print('Analyzing gaps or repetitions \n')
dfs_sort_utils.print_dfs_order_mistakes(kada_cats)

Analyzing gaps or repetitions 

9. Споры о неисполнении или ненадлежащем исполнении обязательств по договорам банковского счета, при осуществлении расчетов --> 9.2. о привлечении к админ. ответственности за недобросовестную конкуренцию
14. Споры о неисполнении или ненадлежащем исполнении обязательств по иным видам договоров --> 16. Корпоративные споры
39. Споры о признании и приведении в исполнение решений иностранных судов и иностранных арбитражных решений --> 50. Споры по делам об оспаривании нормативных правовых актов Президента РФ
53. Споры по делам об оспаривании ненормативных правовых актов Президента РФ --> 54.1. Дела об оспаривании ненормативных правовых актов Совета Федерации
59. споры между высшими органами государственной власти субъектов РФ --> 60.1. жалоба на решение ВККС и ККС о досрочном прекращении полномочий судей
60.5. обращение граждан и организаций на действие (бездействие) судей --> 71. об оспаривании нормативно-правовых актов федер. органов исп. власти в обл. прав

### A decision was taken to delete nodes 60.1 - 60.5 because there are only a few of cases presented in this categories (in sum near 10 cases) and also to delete nodes 54.1 - 54.2 because there are no cases in this categories and they ruin the taxonomy structure.

In [3]:
kada_cats_new = []
print('Deleted ::')
for i, cat in enumerate(kada_cats):
    if cat.startswith('60.') or cat.startswith('54.'):
        print('\t', cat)
    else:
        kada_cats_new.append(cat)

Deleted ::
	 54.1. Дела об оспаривании ненормативных правовых актов Совета Федерации
	 54.2. Дела об оспаривании ненормативных правовых актов Государственной Думы
	 60.1. жалоба на решение ВККС и ККС о досрочном прекращении полномочий судей
	 60.2. жалоба на решение ВККС о наложении дисциплинарных взысканий
	 60.3. жалоба на решение ВККС о результатах квалификационной аттестации
	 60.4. обращение Председателя Верховного Суда Российской Федерации
	 60.5. обращение граждан и организаций на действие (бездействие) судей


In [4]:
dfs_sort_utils.print_dfs_order_mistakes(kada_cats_new)

9. Споры о неисполнении или ненадлежащем исполнении обязательств по договорам банковского счета, при осуществлении расчетов --> 9.2. о привлечении к админ. ответственности за недобросовестную конкуренцию
14. Споры о неисполнении или ненадлежащем исполнении обязательств по иным видам договоров --> 16. Корпоративные споры
39. Споры о признании и приведении в исполнение решений иностранных судов и иностранных арбитражных решений --> 50. Споры по делам об оспаривании нормативных правовых актов Президента РФ
53. Споры по делам об оспаривании ненормативных правовых актов Президента РФ --> 55. Споры по делам об оспаривании ненормативных правовых актов Правительства РФ
59. споры между высшими органами государственной власти субъектов РФ --> 71. об оспаривании нормативно-правовых актов федер. органов исп. власти в обл. правовой охраны рез-ов интеллект. деятельности и средств индивидуализации
72.1.2. об оспаривании ненорм. правовых актов федер. органа исп. власти по селекционным достижениям --> 

No critical errors in taxonomy with deleted nodes. This will be used as the main taxonomy.

## Analyzing intersections of documents between categories

In [5]:
cats = list(data.keys())
for i in range(len(cats)):
    cat1 = cats[i]
    doc1 = set(list(map(lambda x: tuple(x), data[cat1])))
    cat_is_printed = False
    for j in range(i+1, len(cats)):
        cat2 = cats[j]
        doc2 = set(list(map(lambda x: tuple(x), data[cat2])))
        
        intersection = len(doc1.intersection(doc2))
        
        index1 = cat1.split('. ')[0]
        index2 = cat2.split('. ')[0]
        if intersection > 1 and (not index2.startswith(index1)):
            if not cat_is_printed:
                print(index1,'::',len(doc1))
            cat_is_printed = True
            print('\t', index1, '--> ',index2, '::' , intersection)

8 :: 117
	 8 -->  9 :: 25
	 8 -->  16.9 :: 25
	 8 -->  24.2.3 :: 25
	 8 -->  28 :: 25
	 8 -->  29.3 :: 12
	 8 -->  29.9 :: 25
	 8 -->  36 :: 22
	 8 -->  36.3 :: 14
9 :: 106
	 9 -->  16.9 :: 25
	 9 -->  24.2.3 :: 25
	 9 -->  28 :: 25
	 9 -->  29.3 :: 12
	 9 -->  29.9 :: 25
	 9 -->  36 :: 22
	 9 -->  36.3 :: 14
16.9 :: 106
	 16.9 -->  24.2.3 :: 25
	 16.9 -->  28 :: 25
	 16.9 -->  29.3 :: 12
	 16.9 -->  29.9 :: 25
	 16.9 -->  36 :: 22
	 16.9 -->  36.3 :: 14
20.3.1 :: 100
	 20.3.1 -->  34.4 :: 6
20.4 :: 100
	 20.4 -->  34.5 :: 100
20.4.1 :: 100
	 20.4.1 -->  34.5 :: 100
20.5 :: 98
	 20.5 -->  34.1 :: 12
	 20.5 -->  34.2 :: 6
	 20.5 -->  34.3 :: 22
20.5.1 :: 100
	 20.5.1 -->  34.1 :: 17
20.5.2 :: 98
	 20.5.2 -->  34.2 :: 6
20.5.3 :: 100
	 20.5.3 -->  34.3 :: 22
24.2 :: 99
	 24.2 -->  24.5 :: 87
	 24.2 -->  24.6 :: 4
24.2.2 :: 25
	 24.2.2 -->  24.5 :: 24
24.2.3 :: 99
	 24.2.3 -->  24.5 :: 71
	 24.2.3 -->  24.6 :: 3
	 24.2.3 -->  28 :: 25
	 24.2.3 -->  29.3 :: 12
	 24.2.3 -->  29.9 :: 25
	 24

In [6]:
same_groups = [('50', '51', '52',), ('20.4', '34.5')]  # inference from previous cell

Analyzing (strange) group of categories with a lot of intersections

In [7]:
strange_group = [
    '8', '9', '16.9', '24.2.3',
    '28', '29.3', '29.9', '36',
    '36.3'
]
strange_documents = {}
for i in range(len(cats)):
    cat = cats[i]
    index = cat.split('. ')[0]
    if index in strange_group:
        for doc_list in data[cat]:
            doc_tuple = tuple(doc_list)
            if doc_tuple not in strange_documents:
                strange_documents[doc_tuple] = [index]
            else:
                strange_documents[doc_tuple].append(index)

max_prints = 10
n_prints = 0
for i, (_, index_list) in enumerate(
                            sorted(
                                strange_documents.items(), key=lambda x: -len(x[1])
                            )
                        ):
    if i == 0: test_doc = _
    if len(index_list) > 1 and n_prints < max_prints:
        print(f'doc_{i+1}  list of categories :: {index_list}')
        n_prints += 1


doc_1  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36', '36.3']
doc_2  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36', '36.3']
doc_3  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36', '36.3']
doc_4  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36', '36.3']
doc_5  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36', '36.3']
doc_6  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36', '36.3']
doc_7  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36', '36.3']
doc_8  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36']
doc_9  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36']
doc_10  list of categories :: ['8', '9', '16.9', '24.2.3', '28', '29.3', '29.9', '36']


In [8]:
test_doc[0]  # all strange docs have anomalous dates

'27.09.2027'

In [11]:
cats = list(data.keys())
for i in range(len(cats)):
    cat1 = cats[i]
    doc1 = set(list(map(lambda x: tuple(x), data[cat1])))
    doc1 = {doc for doc in doc1 if not date_utils.date_is_anomal(doc[0])}
    cat_is_printed = False
    for j in range(i+1, len(cats)):
        cat2 = cats[j]
        doc2 = set(list(map(lambda x: tuple(x), data[cat2])))
        doc2 = {doc for doc in doc2 if not date_utils.date_is_anomal(doc[0])}
        
        intersection = len(doc1.intersection(doc2))
        
        index1 = cat1.split('. ')[0]
        index2 = cat2.split('. ')[0]
        if intersection > 1 and (not index2.startswith(index1)):
            if not cat_is_printed:
                print(index1,'::',len(doc1))
            cat_is_printed = True
            print('\t', index1, '--> ',index2, '::' , intersection)

20.3.1 :: 100
	 20.3.1 -->  34.4 :: 6
20.4 :: 100
	 20.4 -->  34.5 :: 100
20.4.1 :: 100
	 20.4.1 -->  34.5 :: 100
20.5 :: 98
	 20.5 -->  34.1 :: 12
	 20.5 -->  34.2 :: 6
	 20.5 -->  34.3 :: 22
20.5.1 :: 100
	 20.5.1 -->  34.1 :: 17
20.5.2 :: 98
	 20.5.2 -->  34.2 :: 6
20.5.3 :: 100
	 20.5.3 -->  34.3 :: 22
24.2 :: 99
	 24.2 -->  24.5 :: 87
	 24.2 -->  24.6 :: 4
24.2.2 :: 25
	 24.2.2 -->  24.5 :: 24
24.2.3 :: 74
	 24.2.3 -->  24.5 :: 71
	 24.2.3 -->  24.6 :: 3
50 :: 97
	 50 -->  51 :: 97
	 50 -->  52 :: 97
51 :: 97
	 51 -->  52 :: 97


Well most strange intersections come from documents with anomalous dates.

In [11]:
# THIS CELL IS OUTDATED (DECISION CHANGED)
# following nodes are assigned and added manually due to their absence in the list of categories
# _54 = '54. Дела об оспаривании ненормативных правовых актов'
# _60 = '60. Жалоба на решение ВККС'
# dispute_categories = []
# for cat in dispute_categories_raw:
#     dispute_categories.append(cat)
#     if cat.startswith('53. Споры по делам'):
#         dispute_categories.append(_54)
#     elif cat.startswith('59. споры между'):
#         dispute_categories.append(_60)

In [14]:
taxonomy_df_temp = taxonomy_utils.extract_taxonomy_df(kada_cats_new)
root = (ts.reconstruct_tree_from_array(
    taxonomy_df_temp.values.tolist(), use_anytree=False
))

In [16]:
taxonomy_data = []
for node in ts.preorder_iterator(root):
    if node is root:
        continue
    taxonomy_data.append((node.ID, node.parent.ID, node.index.strip('.'), node.name, node.depth))
taxonomy_df = pd.DataFrame(taxonomy_data, columns=['ID', 'ancestorID', 'cCode', 'Descr', 'depth'])
taxonomy_df.to_csv('data/taxonomies/taxonomy_df.csv', index=False)

Introducing a mapping from old ID's to new Id's

In [17]:
oldID2newID = {}
old_categs_info = pd.read_csv('temp_data/categs_info_old.csv', index_col=0)
for _, row in old_categs_info.iterrows():
    cCode_mask = taxonomy_df['cCode']==row['cCode']
    if cCode_mask.sum() == 1:
        old_id = row['ID']
        new_id = taxonomy_df[cCode_mask]['ID'].values[0]
        oldID2newID[int(old_id)] = int(new_id)
    else:
        print(*row)
        
with open('temp_data/oldID2newID_dict.json', 'w') as f:
    json.dump(oldID2newID, f, )

205 0 Нет Нет 8


<span style="font-family:Papyrus; font-size:1.5em;">EDA of taxonomy modified by lawyers.</span>

In [18]:
df_raw = pd.read_excel('data/taxonomies/Категории КАД.xlsx', engine='openpyxl')
raw_indices = df_raw['Index (по КАД)'].dropna().tolist()
sorted_raw_indices = dfs_sort_utils.dfs_sort(raw_indices)
dfs_sort_utils.print_dfs_order_mistakes(sorted_raw_indices)

9 --> 9.2
14 --> 16
21.1 --> 21.1
21.1.1 --> 21.1.1
22 --> 22.2
25.1 --> 25.1.3.1
28 --> 28
39 --> 50
53 --> 54.1
59 --> 60.1
60.5 --> 71
72.1.2 --> 72.1.2.2


<span style="font-family:Papyrus; font-size:1.5em;">EDA of torrent categories</span>

In [19]:
categs_df = pd.read_csv('temp_data/categs_info_old.csv', index_col=0)
categs_s = dfs_sort_utils.dfs_sort(
    categs_df['cCode'].to_list()[:-1]
)

dfs_sort_utils.print_dfs_order_mistakes(categs_s)

14 --> 16
39 --> 50
53 --> 55
